In [1]:
import stefan

In [2]:
bingo = stefan.Bingo.from_input("../input.txt")

In [3]:
stefan.find_first_winner_score(bingo)

38594

In [4]:
stefan.find_last_winner_score(bingo)

21184

In [259]:
find_last_winner_score(all_boards)

21184